<a href="https://colab.research.google.com/github/Raniaaaaa8/MarBert_tf/blob/main/TRY1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
#Installing the huggingface transformers library
!pip3 install transformers
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.5 MB/s 
     |████████████████████████████████| 86 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 38.5 MB/s 
     |████████████████████████████████| 140 kB 45.7 MB/s 
     |████████████████████████████████| 127 kB 55.1 MB/s 
     |████████████████████████████████| 144 kB 38.9 MB/s 
     |███████████████

# Imports

In [2]:
!pip3 install tensorflow 
import tensorflow as tf
import re
from transformers import TFAutoModelForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

#Importing the necessary modules
from tensorflow import feature_column
from tensorflow.keras.utils import to_categorical


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()



from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TFAutoModelForSequenceClassification


from transformers import create_optimizer

from datasets import list_datasets, load_dataset, Dataset
from pprint import pprint
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split



# Downloading Marbert

In [5]:

#downloading Marbert pretrained model
model = TFAutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT")
model.summary()

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  162841344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 162,842,882
Trainable params: 162,842,882
Non-trainable params: 0
_________________________________________________________________


In [6]:
from transformers import AutoTokenizer
#getting the MarBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
sentence="رانبا هالووو انا خدبج"
token=tokenizer.tokenize(sentence)
print(token)

['ران', '##با', 'هال', '##وو', '##و', 'انا', 'خد', '##بج']


In [10]:
#scikit learn splitting function:
from sklearn.model_selection import train_test_split

# Loading Data


In [11]:
#reading ArSarcasm_train set
df = pd.read_csv("./sample_data/ArSarcasm_train.csv")

# Removing un-needed feilds
df= df[["tweet", "sarcasm"]] 
df.head(20)


,tweet,sarcasm
0,"""نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...",False
1,"""#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...",False
2,"""@Alito_NBA اتوقع انه بيستمر""",False
3,"""@KSA24 يعني ""بموافقتنا"" لأن دمشق صايرة موسكو""",True
4,"""RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...",False
5,شوال الفلوس سويرس مشغول اوى اليومين دول بقناة ...,True
6,"""الأمين العام للأمم المتحدة: بشار الأسد قتل 30...",False
7,"حسبنا الله ونعم الوكيل "" #انتخبوا_العرص #بيقول...",True
8,"""RT @isharatidriss: ٣٠)#امريكا_ومحاربة_الارهاب...",False
9,"""رئيس لجنة الحكام رضا البلتاجى يفتح النار على ...",False


# Cleaning Data

In [12]:
# data cleaning
def clean_data(text): 
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("آ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")
    text = text.replace("#", " ");
    text = text.replace("@", " ");
    text = text.replace("_", " ");
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [13]:
#apply data cleaning for tweet column
df["tweet"] = df.tweet.apply(clean_data)
df.head(15)

,tweet,sarcasm
0,نصيحه ما عمرك اتنزل لعبه سوبر ماريو مش زي ما ك...,False
1,نادين نسيب نجيم مجله ماري كلير ملكه الصحراء,False
2,اتوقع انه بيستمر,False
3,يعني بموافقتنا لان دمشق صايره موسكو,True
4,قايد في الحرس يعترف بفقدان السيطره الامنيه في ...,False
5,شوال الفلوس سويرس مشغول اوي اليومين دول بقناه ...,True
6,الامين العام للامم المتحده بشار الاسد قتل الف ...,False
7,حسبنا الله ونعم الوكيل انتخبوا العرص بيقولك اع...,True
8,امريكا ومحاربه الارهاب الذي صنعته لمواصله الكي...,False
9,رييس لجنه الحكام رضا البلتاجي يفتح النار علي ع...,False


In [14]:
df["label"]=df["sarcasm"]


df.drop(df[df.tweet == '[]'].index, inplace=True, axis=0)
df.dropna(inplace=True)

df.label = LabelEncoder().fit_transform(df.label)
df_train=df
#df_train, df_val = train_test_split(df, test_size=0.05,  shuffle=True)

#df_val.to_csv('val.csv')
#df_train.head(10)
#print(df_val)
#df.groupby(["label"]).agg(["count"])
df_train.groupby(["label"]).agg(["count"])


,tweet,sarcasm
,count,count
label,,
0,7100,7100
1,1337,1337


In [15]:
print (df_train)

                                                  tweet  sarcasm  label
0     نصيحه ما عمرك اتنزل لعبه سوبر ماريو مش زي ما ك...    False      0
1           نادين نسيب نجيم مجله ماري كلير ملكه الصحراء    False      0
2                                      اتوقع انه بيستمر    False      0
3                   يعني بموافقتنا لان دمشق صايره موسكو     True      1
4     قايد في الحرس يعترف بفقدان السيطره الامنيه في ...    False      0
...                                                 ...      ...    ...
8432  توصيات تويتسو تطبيق يوتيوب الاطفال من جوجل يعر...    False      0
8433  سي بي سيابراهيم محلب يشارك باحتفاليه تنزانيا ا...    False      0
8434  كلب اسراييل وسيفها الارهابي بوتن قاتل النساء و...     True      1
8435  احمد فتفت صار مغضوب عليه بمجرد اعترضو علي ميشا...     True      1
8436  قناه اسيا مندوب سوريا في الامم المتحده بشار ال...    False      0

[8437 rows x 3 columns]


In [16]:
df_train[["label", "tweet"]].to_csv("df_train.csv", encoding="utf-8-sig", index=False)
from datasets import load_dataset

df = load_dataset('csv', data_files=["./df_train.csv"], delimiter=",", split="train")
df = df.train_test_split()
df

Using custom data configuration default-600f18fbcee4c8cb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-600f18fbcee4c8cb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['label', 'tweet'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['label', 'tweet'],
        num_rows: 2110
    })
})

# Marbert tokenizer

In [17]:

#create our function for features tokenization 
def tokenize_function(examples):
    return tokenizer(examples["tweet"], truncation=True)

tokenized_df = df.map(tokenize_function, batched=True)
tokenized_df = tokenized_df.remove_columns(["tweet"]) 
print(tokenized_df)



  0%|          | 0/7 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2110
    })
})


In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [19]:
tf_train_dataset = tokenized_df["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
print(tf_train_dataset)
tf_validation_dataset = tokenized_df["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

<PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(16,), dtype=tf.int64, name=None)}>


In [21]:
#f_train_dataset.visualization.Visualiser

In [22]:
from transformers import create_optimizer
batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_df["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [23]:
#lenth=len(df)
#print(model.config.num_labels)
#print (lenth)
model = TFAutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

callbacks = [model_checkpoint_callback]

# Optimizer

In [37]:
optimizer= tf.keras.optimizers.Adam(learning_rate=0.001)


In [26]:
#model.compile(
#    optimizer=tf.keras.optimizers.SGD(learning_rate=2e-4),
#    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#    metrics=[tf.metrics.SparseCategoricalAccuracy(), ]
#)

In [27]:

print(len(tf_train_dataset))

395


# Training

In [ ]:
model.compile(optimizer=optimizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=1
)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


170/395 [===========>..................] - ETA: 25:22 - loss: 0.4589

In [ ]:
# model.save_pretrained("./")
model.load_weights("./tf_marbert_model.h5")

In [ ]:
# predict_input = tokenizer.encode(df_cl["text"][0],
#                                  truncation=True,
#                                  padding=True,
#                                  return_tensors="tf")

# Test and evaluation

In [30]:
#reading ArSarcasm_train set
df_test = pd.read_csv("./sample_data/ArSarcasm_test.csv")

# Removing un-needed feilds
df_test= df_test[["tweet", "sarcasm"]] 

In [31]:
#apply data cleaning for tweet column
df_test["tweet"] = df_test.tweet.apply(clean_data)

In [32]:
df_test["label"]=df_test["sarcasm"]


df_test.drop(df_test[df_test.tweet == '[]'].index, inplace=True, axis=0)
df_test.dropna(inplace=True)

df_test.label = LabelEncoder().fit_transform(df_test.label)

df_test.groupby(["label"]).agg(["count"])

,tweet,sarcasm
,count,count
label,,
0,1765,1765
1,345,345


In [33]:
input_seq_test = [tokenizer.encode(lst, truncation=True, padding=True, return_tensors="tf") 
                      for lst in df_test["tweet"]]

In [36]:
df_test

,tweet,sarcasm,label
0,هههههه حزب الله حطموا اسطوره الميركافا الاسراي...,True,1
1,البنات اللي م صامو بقولكم ترا رمضان قرب والوقت...,False,0
2,اشاره رابعه اشبه بنار تحرق الانقلابين,True,1
3,ماهي مميزات درجه بزنس علما اني في بروكسيل واري...,False,0
4,ما لا تراه علي التلفاز منافسه شديده بين المرشح...,False,0
...,...,...,...
2105,د علاء الاسواني ما هي اليات التحول الديموقراطي...,False,0
2106,اوقفوا الفحم في مصر,False,0
2107,الحب هو ان تشبع حبيبك اهتماما قبل اشباع رغباتك,False,0
2108,اللهم كن لاخواننا في سورياستمايه الف قتيلوقيل ...,False,0


In [34]:
len(input_seq_test)

2110

In [35]:
input_seq_test[0]

<tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[    2,  2372,  8709,  8709,  5167,  1944, 41538,  1958, 25330,
        47931,  3986,  2355, 11719,  1021,  2504,  6233,  4742,  2643,
        54787,  1947, 13126, 39693,     3]], dtype=int32)>

In [ ]:
tf_output = [np.argmax(model.predict(lst)[0], axis=1) for lst in input_seq_test]

In [ ]:
prediction = [lst[0] for lst in tf_output]